Ön hazırlık (kütüphane kurulumu)

In [2]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.5 MB/s eta 0:00:00


GROVER ARAMA ALGORİTMASI

Cirq ile Grover Arama Örneği

In [3]:
import cirq
import numpy as np

# 3 qubit + 3 ancilla (oracle için) qubitleri
qubits = cirq.LineQubit.range(3)

# Aranan değer 3 -> |011>
marked_state = '011'

# Oracle: aranan durumu -1 fazına çevirir
def grover_oracle(qubits, marked_state):
    circuit = cirq.Circuit()
    for i, bit in enumerate(marked_state):
        if bit == '0':
            circuit.append(cirq.X(qubits[i]))
    circuit.append(cirq.Z(cirq.ControlledGate(cirq.Z, num_controls=len(qubits)-1).on(*qubits)))
    for i, bit in enumerate(marked_state):
        if bit == '0':
            circuit.append(cirq.X(qubits[i]))
    return circuit

# Diffusion operator (W)
def diffusion_operator(qubits):
    circuit = cirq.Circuit()
    circuit.append(cirq.H.on_each(*qubits))
    circuit.append(cirq.X.on_each(*qubits))
    circuit.append(cirq.H(qubits[-1]))
    circuit.append(cirq.CCX(qubits[0], qubits[1], qubits[2]))
    circuit.append(cirq.H(qubits[-1]))
    circuit.append(cirq.X.on_each(*qubits))
    circuit.append(cirq.H.on_each(*qubits))
    return circuit

# Devre oluşturma
circuit = cirq.Circuit()
circuit.append(cirq.H.on_each(*qubits))  # Başlangıç Hadamard
# Grover iterasyonu ~ 2 kez (n=3)
for _ in range(2):
    circuit.append(grover_oracle(qubits, marked_state))
    circuit.append(diffusion_operator(qubits))

# Ölçüm
circuit.append(cirq.measure(*qubits, key='result'))

# Simülasyon
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print(result)


AttributeError: 'ControlledOperation' object has no attribute 'dimension'

Qiskit ile Grover Arama Örneği

In [4]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import ZGate

n = 3
marked_state = '011'

qc = QuantumCircuit(n, n)

# Başlangıç: Hadamard
qc.h(range(n))

# Oracle
qc.x([0])  # qubit 0'ı tersle (marked_state '0')
qc.h(2)
qc.ccx(0,1,2)
qc.h(2)
qc.x([0])
# Diffusion
qc.h(range(n))
qc.x(range(n))
qc.h(2)
qc.ccx(0,1,2)
qc.h(2)
qc.x(range(n))
qc.h(range(n))

# Ölçüm
qc.measure(range(n), range(n))

simulator = Aer.get_backend('qasm_simulator')
result = execute(qc, simulator, shots=20).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Grover Arama Örneği

In [5]:
import numpy as np
from qutip import *

# 3 qubit
n = 3
N = 2**n

# Aranan durum |011> -> decimal 3
marked_index = 3

# Başlangıç durumu: |000> süperpozisyon
psi0 = tensor([basis(2,0) for _ in range(n)])
# Hadamard operatörü
H = (1/np.sqrt(2)) * np.array([[1,1],[1,-1]])
Hn = tensor([Qobj(H) for _ in range(n)])
psi = Hn * psi0

# Oracle: aranan durumu -1 fazına çevir
Uf = Qobj(np.identity(N))
Uf[marked_index, marked_index] = -1

# Diffusion operator (W)
psi_uniform = (1/np.sqrt(N))*sum([tensor([basis(2,i>>j & 1) for j in range(n-1,-1,-1)]) for i in range(N)])
W = 2*ket2dm(psi_uniform) - qeye(N)

# Grover iterasyonu
G = W * Uf

# Kaç iterasyon? ~ pi/4 * sqrt(N)
m = int(np.round(np.pi/4 * np.sqrt(N)))

# Iterasyon uygulama
for _ in range(m):
    psi = G * psi

# Ölçüm olasılıkları
probs = np.abs(psi.full().flatten())**2
for i, p in enumerate(probs):
    print(f"State |{i:03b}> : Probability {p:.3f}")


TypeError: 'Qobj' object does not support item assignment

AMPLITUDE AMPLIFICATION (GENLİK BÜYÜTME)

Cirq Örneği

In [6]:
import cirq
import numpy as np

# 3 qubit kullanacağız
qubits = [cirq.LineQubit(i) for i in range(3)]

# Aranan durum |011>
target_state = '011'

# Oracle: Aranan durumu -1 faz ile işaretle
def grover_oracle():
    circuit = cirq.Circuit()
    for i, bit in enumerate(target_state):
        if bit == '0':
            circuit.append(cirq.X(qubits[i]))
    circuit.append(cirq.CCX(qubits[0], qubits[1], qubits[2]))  # basit oracle
    for i, bit in enumerate(target_state):
        if bit == '0':
            circuit.append(cirq.X(qubits[i]))
    return circuit

# Diffusion operatörü
def diffusion_operator():
    circuit = cirq.Circuit()
    circuit.append([cirq.H(q) for q in qubits])
    circuit.append([cirq.X(q) for q in qubits])
    circuit.append(cirq.H(qubits[-1]))
    circuit.append(cirq.CCX(qubits[0], qubits[1], qubits[2]))
    circuit.append(cirq.H(qubits[-1]))
    circuit.append([cirq.X(q) for q in qubits])
    circuit.append([cirq.H(q) for q in qubits])
    return circuit

# Başlangıç devresi
circuit = cirq.Circuit()
circuit.append([cirq.H(q) for q in qubits])

# Grover iterasyonu sayısı (yaklaşık √N)
num_iterations = 1
for _ in range(num_iterations):
    circuit.append(grover_oracle())
    circuit.append(diffusion_operator())

# Ölçüm
circuit.append([cirq.measure(q) for q in qubits])

# Simülasyon
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print(result)


q(0)=11010100101001111111
q(1)=10011000111101111100
q(2)=01011111111100001001


Qiskit Örneği

In [7]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

# 3 qubitlik sistem
qc = QuantumCircuit(3, 3)

# Başlangıç: Süperpozisyon
qc.h([0,1,2])

# Oracle: |011> durumunu işaretleme
qc.x(0)
qc.ccx(0,1,2)
qc.x(0)

# Diffusion operatörü
qc.h([0,1,2])
qc.x([0,1,2])
qc.h(2)
qc.ccx(0,1,2)
qc.h(2)
qc.x([0,1,2])
qc.h([0,1,2])

# Ölçüm
qc.measure([0,1,2],[0,1,2])

# Simülasyon
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
result = job.result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP Örneği

In [8]:
from qutip import *
import numpy as np

# 3 qubitlik durum
n = 3
N = 2**n
# Başlangıç: eşit süperpozisyon
psi = tensor([basis(2,0) for _ in range(n)])
psi = (tensor([basis(2,0)+basis(2,1) for _ in range(n)])).unit()

# Oracle: |011> durumuna -1 faz
target_index = 3  # |011> = 3
oracle = qeye(N)
oracle[target_index,target_index] = -1

# Difüzyon: yansıtma operatörü
psi0 = psi.full()
diffusion = 2 * (ket2dm(psi)) - qeye(N)

# Grover iterasyonu
G = diffusion * oracle

# Bir iterasyon
psi_new = G * psi
print("Durum vektörü:\n", psi_new)


TypeError: 'Qobj' object does not support item assignment

QUANTUM WALKS (KUANTUM YÜRÜYÜŞLER)


Cirq ile Ayrık Zamanlı Kuantum Yürüyüş

In [9]:
import cirq
import numpy as np

# 3 pozisyon qubit ve 1 coin qubit
coin = cirq.LineQubit(0)
positions = [cirq.LineQubit(i+1) for i in range(3)]

# Başlangıç: coin |0> ve pozisyon |010>
circuit = cirq.Circuit()
circuit.append(cirq.X(positions[1]))  # başlangıç pozisyon 1
circuit.append(cirq.H(coin))  # coin süperpozisyon

# Adım fonksiyonu: coin |0> -> sola, coin |1> -> sağa
def quantum_walk_step():
    step_circuit = cirq.Circuit()
    for i, pos in enumerate(positions):
        # kontrol: coin qubit
        if i > 0:
            step_circuit.append(cirq.CNOT(coin, positions[i-1]))
        if i < len(positions)-1:
            step_circuit.append(cirq.CNOT(coin, positions[i+1]))
    return step_circuit

# 2 adım yürüyüş
for _ in range(2):
    circuit.append(quantum_walk_step())
    circuit.append(cirq.H(coin))  # coin yeniden Hadamard

# Ölçüm
circuit.append([cirq.measure(q) for q in positions+ [coin]])

simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print(result)


q(0)=00010001001111000111
q(1)=11101110110000111000
q(2)=11111111111111111111
q(3)=11101110110000111000


Qiskit ile Ayrık Zamanlı Kuantum Yürüyüş

In [10]:
from qiskit import QuantumCircuit, Aer, execute

# 1 coin qubit, 3 pozisyon qubit
qc = QuantumCircuit(4,4)
qc.x(2)  # başlangıç pozisyon |010>
qc.h(0)  # coin süperpozisyon

# 1 adım yürüyüş: coin |0> -> sola, coin |1> -> sağa
qc.cx(0,1)  # coin control -> pozisyon 0
qc.cx(0,3)  # coin control -> pozisyon 3

# 2. adım
qc.h(0)
qc.cx(0,1)
qc.cx(0,3)

qc.measure([0,1,2,3],[0,1,2,3])

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1024).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Ayrık Zamanlı Kuantum Yürüyüş

In [11]:
from qutip import *
import numpy as np

# Coin ve pozisyon: 1 coin, 3 pozisyon (toplam 2*3 = 6 boyutlu Hilbert)
coin = basis(2,0)
pos = basis(3,1)  # başlangıç pozisyon |1>
psi = tensor(coin,pos)

# Coin Hadamard operatörü
H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])

# Kaydırma operatörü: coin |0> sola, coin |1> sağa
def shift_operator():
    S = qzero(6,6)
    # coin |0> sola
    S += tensor(basis(2,0)*basis(2,0).dag(), qdiags([0,1,0],[0],3))
    # coin |1> sağa
    S += tensor(basis(2,1)*basis(2,1).dag(), qdiags([0,1,0],[0],3))
    return S

# 1 adım: coin + shift
psi = tensor(H, qeye(3)) * psi  # coin Hadamard
S = shift_operator()
psi = S * psi

print("Durum vektörü:\n", psi)


TypeError: object of type 'int' has no len()

QUANTUM APPROXIMATE OPTIMIZATION ALGORİTMASI (QAOA)

Cirq ile QAOA (Max-Cut Örneği)

In [12]:
import cirq
import numpy as np

# Basit 2 düğümlü grafik: 0 -- 1
qubits = [cirq.GridQubit(0, i) for i in range(2)]
gamma = np.pi/4
beta = np.pi/4

# Başlangıç devresi: tüm qubitler |+> durumunda
circuit = cirq.Circuit([cirq.H(q) for q in qubits])

# Problem Hamiltonyeni uygulaması (CPhase ile kenar ağırlığı)
circuit.append(cirq.CZ(qubits[0], qubits[1])**(gamma/np.pi))

# Mixer Hamiltonyeni (Pauli-X rotasyonu)
circuit.append([cirq.rx(2*beta)(q) for q in qubits])

# Ölçüm
circuit.append([cirq.measure(q) for q in qubits])

# Simülasyon
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print(result)


q(0, 0)=00000111001000001011
q(0, 1)=10100000000111011101


Qiskit ile QAOA (Max-Cut Örneği)

In [13]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

# 2 qubitli devre
qc = QuantumCircuit(2,2)
gamma = np.pi/4
beta = np.pi/4

# Başlangıç: tüm qubitler |+>
qc.h([0,1])

# Problem Hamiltonyeni: CZ^gamma
qc.cz(0,1)
qc.rz(2*gamma, 1)  # Gamma etkisi
qc.cz(0,1)

# Mixer Hamiltonyeni
qc.rx(2*beta, 0)
qc.rx(2*beta, 1)

# Ölçüm
qc.measure([0,1],[0,1])

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1024).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile QAOA (Max-Cut Örneği)

In [14]:
from qutip import *
import numpy as np

# 2 qubit
n = 2
psi = tensor(basis(2,0), basis(2,0))  # |00>
H_plus = (1/np.sqrt(2))*(sigmax() + qeye(2))

# Başlangıç |+>^n
psi = tensor(H_plus*basis(2,0), H_plus*basis(2,0))

# Problem Hamiltonyeni (Max-Cut): H_C = Z0 Z1
H_C = tensor(sigmaz(), sigmaz())

# Mixer Hamiltonyeni: H_B = X0 + X1
H_B = tensor(sigmax(), qeye(2)) + tensor(qeye(2), sigmax())

gamma = np.pi/4
beta = np.pi/4

# QAOA durumu: |psi> = e^{-i beta H_B} e^{-i gamma H_C} |+>^n
U_C = (-1j*gamma*H_C).expm()
U_B = (-1j*beta*H_B).expm()
psi_final = U_B * U_C * psi

# Ölçüm olasılıkları
probabilities = np.abs(psi_final.full())**2
print("Durum olasılıkları:\n", probabilities)


Durum olasılıkları:
 [[0.25]
 [0.25]
 [0.25]
 [0.25]]


VARIATIONAL QUANTUM EIGENSOLVER (VQE)

Cirq ile VQE (2-Qubit Örneği)

In [16]:
import cirq
import numpy as np
from scipy.optimize import minimize

# 2 qubit tanımla
qubits = cirq.LineQubit.range(2)

# Basit Hamiltonyen: H = Z0 + Z1 + Z0*Z1
def hamiltonian_expectation(params):
    circuit = cirq.Circuit()
    # Parametrik ansatz: Rx rotasyonları
    circuit.append(cirq.rx(params[0])(qubits[0]))
    circuit.append(cirq.rx(params[1])(qubits[1]))
    # Ölçüm
    circuit.append(cirq.measure(*qubits, key='m'))

    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=1000)
    counts = result.histogram(key='m')

    # Beklenen enerji (basitleştirilmiş)
    energy = 0
    for state, count in counts.items():
        z0 = 1 if (state & 1) == 0 else -1
        z1 = 1 if (state & 2) == 0 else -1
        energy += count * (z0 + z1 + z0*z1)
    energy /= 1000
    return energy

# Başlangıç parametreleri
init_params = np.random.rand(2)

# Klasik optimizasyon
res = minimize(hamiltonian_expectation, init_params)
print("En düşük enerji:", res.fun)
print("Optimum parametreler:", res.x)


En düşük enerji: 2.34
Optimum parametreler: [0.62623452 0.50183124]


Qiskit ile VQE Örneği

In [17]:
from qiskit import Aer, execute
from qiskit.circuit import QuantumCircuit, Parameter
from scipy.optimize import minimize
import numpy as np

# Parametreleri tanımla
theta0 = Parameter('θ0')
theta1 = Parameter('θ1')

# Ansatz devresi
qc = QuantumCircuit(2)
qc.rx(theta0, 0)
qc.rx(theta1, 1)
qc.measure_all()

# Hamiltonyen: H = Z0 + Z1 + Z0*Z1
def hamiltonian_expectation(params):
    bound_qc = qc.bind_parameters({theta0: params[0], theta1: params[1]})
    backend = Aer.get_backend('qasm_simulator')
    result = execute(bound_qc, backend, shots=1000).result()
    counts = result.get_counts()
    energy = 0
    for state, count in counts.items():
        z0 = 1 if state[-1] == '0' else -1
        z1 = 1 if state[-2] == '0' else -1
        energy += count * (z0 + z1 + z0*z1)
    energy /= 1000
    return energy

init_params = np.random.rand(2)
res = minimize(hamiltonian_expectation, init_params)
print("En düşük enerji:", res.fun)
print("Optimum parametreler:", res.x)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile VQE Örneği

In [18]:
import qutip as qt
import numpy as np
from scipy.optimize import minimize

# 2 qubit Hamiltonyen: H = Z0 + Z1 + Z0*Z1
Z0 = qt.tensor(qt.sigmaz(), qt.qeye(2))
Z1 = qt.tensor(qt.qeye(2), qt.sigmaz())
H = Z0 + Z1 + Z0*Z1

# Parametrik ansatz: |ψ(θ)> = Rx(θ0)⊗Rx(θ1)|00>
def ansatz(params):
    rx0 = (-1j * params[0] * qt.sigmax()).expm()
    rx1 = (-1j * params[1] * qt.sigmax()).expm()
    psi0 = qt.tensor(qt.basis(2,0), qt.basis(2,0))
    return rx0 * rx1 * psi0

def energy_expectation(params):
    psi = ansatz(params)
    return qt.expect(H, psi)

init_params = np.random.rand(2)
res = minimize(energy_expectation, init_params)
print("En düşük enerji:", res.fun)
print("Optimum parametreler:", res.x)


TypeError: incompatible dimensions [[2], [2]] and [[2, 2], [1]]